# BirdCLEF2022

## 比赛简介

这个比赛主要任务是对音频中的鸟叫声进行识别，测试时会给定音频要求判断给定时间段中是否有某种特定鸟类的叫声。

请认真阅读比赛要求指南：[比赛首页](https://www.kaggle.com/c/birdclef-2022)

## 关于本notebook

本notebook会对所给数据集进行基本分析及预处理，将音频文件进行转换，做简单的特征工程操作。

## 进度

 - [x] 数据集文件总览
 - [x] 音频样例分析
 - [x] 音频转换
 - [x] training set预处理
 
## 一些参考的文章及notebook

[https://www.kaggle.com/jirkaborovec/birdclef-eda-baseline-flash-efficientnet/data](https://www.kaggle.com/jirkaborovec/birdclef-eda-baseline-flash-efficientnet/data)

[https://www.kaggle.com/hasanbasriakcay/birdclef22-eda-noise-reduction](https://www.kaggle.com/hasanbasriakcay/birdclef22-eda-noise-reduction)

[https://www.kaggle.com/drcapa/birdclef-2022-starter](https://www.kaggle.com/drcapa/birdclef-2022-starter)

感谢这些作者做出的贡献

# Import libraries

In [ ]:
# base
import os
import json
import random
import pandas as pd
import numpy as np
import torch

# plot
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

# For exploring audio files
import librosa
import librosa.display
import torchaudio
import IPython.display as ipd

# tqdm
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

# 数据总览

## 所有文件

In [ ]:
base_dir = '../input/birdclef-2022/'
os.listdir(base_dir)

## 1.test_soundscapes文件

里面存放要识别的soundscape文件，下面是可下载数据集的样例

In [ ]:
ipd.Audio('../input/birdclef-2022/test_soundscapes/soundscape_453028782.ogg')

## 2.train_audio

里面存放用于训练的声音，文件夹名就是鸟名

In [ ]:
train_audio_dir = base_dir + 'train_audio/'
print(train_audio_dir + '#')
print(os.listdir(train_audio_dir)[:5])
print()
print(train_audio_dir+'afrsil1'+'#')
print(os.listdir(train_audio_dir+'afrsil1'))

In [ ]:
# afrsil1的一个声音展示
ipd.Audio('../input/birdclef-2022/train_audio/afrsil1/XC125458.ogg')

## 3. eBird_Txonomy_v2021.csv

这个csv展示的时不同物种间的关系表

In [ ]:
ebird_df = pd.read_csv('../input/birdclef-2022/eBird_Taxonomy_v2021.csv')
ebird_df.head()

## 4.sample_submission.csv

提交的样例csv。从该文件可以看出提交格式。

 - row_id: test.csv中的对应主键
 - target: 判断是否有鸟的声音，true/false

**注意：**可下载的数据集是不全的，同样sample_submission.csv文件也是不全的，但当用private数据集测试时sample_submission.csv大小是对的，所以可以通过修改sample来提交正确格式的submission，这个可以通过将sample中的target改为全true进行验证，能得到分数0.51

In [ ]:
sample_sub_df = pd.read_csv('../input/birdclef-2022/sample_submission.csv')
sample_sub_df

## 5.scored_birds.json

一个重要的json文件，里面记录了21种鸟类的名字，这些鸟是需要判断是否出现在测试音频中的鸟类，所以生成submission可以通过遍历scored birds生成true/false而不用遍历所有的test.csv，这可以减少inference过程中load data的时间。

In [ ]:
with open(base_dir+'scored_birds.json') as f:
    scored_birds = json.load(f)

    
print('scored birds:')
print(scored_birds[:5])
print()
print('length:')
print(len(scored_birds))

## 6.test.csv

测试集的元数据。只有前三行可供下载;完整的test.csv在隐藏的测试集中提供。

 - row_id:表主键
 - file_id:该行对应音频
 - bird:要判断的鸟类，只有scored birds中的鸟类，并且判断是每5秒判断一次
 - end_time:该音频窗口的结束时间(5的倍数）

In [ ]:
test_df = pd.read_csv('../input/birdclef-2022/test.csv')
test_df.head()

## 7.train_metadata.csv

所有的元数据。主要有用的数据列如下：

 - primary_label：鸟的名字
 - secondary_labels：由录音师注释的background species
 - author: 提供录音的作者
 - filename: 文件路径
 - rating: 标志的可信度，1最低5最高，0代表目前还没有排名

In [ ]:
train_metadata_df = pd.read_csv('../input/birdclef-2022/train_metadata.csv')
train_metadata_df.head()

# 音频处理样例

下面讲述如何处理音频文件，让其能应用到神经网络及其他类似算法中。主要使用torchaudio库，当然也可以使用librosa库进行音频处理

## import

导入一些必要的包...


In [ ]:
!pip install -q noisereduce

In [ ]:
import torch
import torchaudio
import noisereduce as nr

## 音频读取

通过torchaudio.load读取音频，返回signal和sample rate。由输出可以看出音频是单通道并且length为1920000=60×32k（sample时长为1min）。

In [ ]:
sample_soundscape_dir = '../input/birdclef-2022/test_soundscapes/soundscape_453028782.ogg'
sam_sig, sam_sr = torchaudio.load(sample_soundscape_dir)
print('sigal:\n',sam_sig)
print()
print('shape:\n',sam_sig.shape)
print()
print('sample rate:\n',sam_sr)

## 可视化：

In [ ]:
# y = sam_sig[0]
# x = np.arange(sam_sig.shape[1])
# plt.plot(x,y)

## 转换成双通道

一些声音文件是单声道（即1个音频通道），而大多数则是立体声（即2个音频通道）。由于我们的模型期望所有项目都具有相同的尺寸，因此我们将第一个通道复制到第二个通道，从而将单声道文件转换为立体声。

In [ ]:
  # ----------------------------
  # Convert the given audio to the desired number of channels
  # ----------------------------
  @staticmethod
  def rechannel(aud, new_channel):
    sig, sr = aud

    if (sig.shape[0] == new_channel):
      # Nothing to do
      return aud

    if (new_channel == 1):
      # Convert from stereo to mono by selecting only the first channel
      resig = sig[:1, :]
    else:
      # Convert from mono to stereo by duplicating the first channel
      resig = torch.cat([sig, sig])

    return ((resig, sr))

## 梅尔谱图

一般会将音频转为梅尔普图作为输入加入到深度学习模型中，可以通过如下方法，默认进行了去噪。参考了[https://www.kaggle.com/jirkaborovec/birdclef-eda-baseline-flash-efficientnet/notebook](https://www.kaggle.com/jirkaborovec/birdclef-eda-baseline-flash-efficientnet/notebook)

In [ ]:
def create_spectrogram(fname, reduce_noise: bool = False, max_length = int(1e7), device = "cpu"):
    waveform, sample_rate = torchaudio.load(fname)
    waveform = waveform[0][:max_length]
    transform = torchaudio.transforms.Spectrogram(n_fft=1800, win_length=1024).to(device)
    if reduce_noise:
        waveform = torch.tensor(nr.reduce_noise(y=waveform, sr=sample_rate, win_length=transform.win_length, use_tqdm=False, n_jobs=-1))
    spectrogram = transform(waveform.to(device))
    return torch.log(spectrogram).numpy()

print(sample_soundscape_dir+'#')
sg = create_spectrogram(sample_soundscape_dir, reduce_noise=True)
print(sg)
print()
print('shape:', sg.shape)

In [ ]:
plt.imshow(sg)

# 数据预处理

将所有的音频文件转换为img，需要对上述代码进行重构。将音频裁剪成5s长度，与测试集对齐。注意是否降噪对结果有巨大影响，同时图片的点很多负无穷，因为数值问题，没办法进行很好的归一化，保存时直接截断保存即可


In [ ]:
from torch.utils.data import DataLoader
from math import ceil

SIGNAL_LENGTH = 5 # 5s
SAMPLE_RATE = 32000 # 32kHz

def get_spectrograms(fname, 
                     reduce_noise: bool = False,
                     sample_rate: int = 32000,
                     frame_length: int = 5,
                     device = "cpu"):
    waveform, sample_rate = torchaudio.load(fname)
    transform = torchaudio.transforms.Spectrogram(n_fft=1800, win_length=512).to(device)
    if reduce_noise:
        waveform = torch.tensor(nr.reduce_noise(
            y=waveform, sr=sample_rate, win_length=transform.win_length, use_tqdm=True, n_jobs=2,
        ))
    sig = waveform[0]
    
    # Split signal into five second chunks
    
    sig_splits = []
    spectrograms = []
#     print(waveform)
#     print(sig)
#     print(len(sig))
#     print(waveform.size()[-1])
#     print(sig[:int(frame_length * sample_rate)])
#     print(waveform[0][:int(frame_length * sample_rate)])
    
    for i in range(0, len(sig), int(frame_length * sample_rate)):
        split = sig[i:i + int(frame_length * sample_rate)]
 
        # End of signal?
        if len(split) < int(frame_length * sample_rate):
            break
        
        frame = split
#         sg = torchaudio.transforms.AmplitudeToDB(top_db=80)(waveform[:][i:i + int(frame_length * sample_rate)])
#         np.array(sg)
 
        db = torch.log(transform(frame.to(device)))
#         db[db<-80]=-80
        
        sg = np.nan_to_num(db.cpu().numpy())
#         sg -= sg.min()
#         sg /= sg.max()
        spectrograms.append(sg)
    
    return spectrograms



下面展示不去噪的结果图片

In [ ]:
# sample for test
sgs = get_spectrograms(sample_soundscape_dir, reduce_noise=False)

fig, axarr = plt.subplots(ncols=len(sgs), figsize=(4 * len(sgs), 4))
for i, sg in enumerate(sgs):
    ax = axarr[i].imshow(sg, vmin=-80, vmax=20)
plt.colorbar(ax)
fig.tight_layout()

上述结果是未去噪的结果图像，下面将展示去噪后的图像。可以看出去噪对梅尔普图影响显著。

In [ ]:
sgs = get_spectrograms(sample_soundscape_dir, reduce_noise=True)

fig, axarr = plt.subplots(ncols=len(sgs), figsize=(4 * len(sgs), 4))
for i, sg in enumerate(sgs):
    ax = axarr[i].imshow(sg, vmin=-80, vmax=20)
plt.colorbar(ax)
fig.tight_layout()

接下来只需要将其运用到所有training set中即可。